<a href="https://colab.research.google.com/github/Ugo5738/Natural_language_processing_projects/blob/main/Deploying_Sentiment_Analysis_with_Tensorflow_Serving(tfds_dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass

In [ ]:
import os
import json
import tempfile
import requests
import numpy as np

import tensorflow as tf

print("\u2022 using Tensorflow version:", tf.__version__)

• using Tensorflow version: 2.4.1


# Handling Dataset

## Create Dataset

In [ ]:
import tensorflow_datasets as tfds
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True) 

# as_supervised if true will have the tuple of input and label
# if True, tfds.load will return the tuple (tf.data.Dataset, tfds.core.DatasetInfo), the latter containing the info associated with the builder.

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0HTVON/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0HTVON/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0HTVON/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_data, test_data = imdb['train'], imdb['test']

In [ ]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
    training_sentences.append(s.numpy().decode('utf-8'))
    training_labels.append(l.numpy())

    testing_sentences.append(s.numpy().decode('utf-8'))
    testing_labels.append(l.numpy())

In [ ]:
print(training_sentences[20])

I can still remember first seeing this on TV. I couldn't believe TVNZ let it on! I had to own it! A lot of the humor will be lost on non-NZ'ers, but give it a go! <br /><br />Since finishing the Back of the Y series Matt and Chris have gone on to bigger and better(?) things. NZ's greatest dare-devil stuntman, Randy Campbell has often appeared on the British TV series Balls of Steel. Yes, he still f^@ks up all his stunts because he is too drunk.<br /><br />Also the 'house band' Deja Voodoo have since released 2 albums, Brown Sabbath and Back in Brown. The band consists of members of the Back of the Y team and singles such as 'I Would Give You One of My Beers (But I've Only Got 6)' and 'You Weren't Even Born in The 80's' continue their humor.<br /><br />The South-By-Southwest film festival also featured their feature length film 'The Devil Made Me Do It' which will be released early 2008 in NZ.<br /><br />All up, if you don't find these guys funny then you can just F%^K OFF!!


In [ ]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [ ]:
training_labels_final[:20]

array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0])

In [ ]:
vocab_size = 100000
embedding_dim = 16
max_length = 250
trunk_type = 'post'
oov_token = '<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
training_sequence = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequence, maxlen=max_length, truncating=trunk_type)

testing_sequence = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequence, maxlen=max_length)

# Build and Train model

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 16)           1600000   
_________________________________________________________________
flatten (Flatten)            (None, 4000)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 24006     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 1,624,013
Trainable params: 1,624,013
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
num_epochs = 10

model.fit(training_padded, 
          training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded, testing_labels_final),
          verbose=2)

Epoch 1/10
782/782 - 11s - loss: 0.1464 - accuracy: 0.9492 - val_loss: 0.0967 - val_accuracy: 0.9693
Epoch 2/10
782/782 - 11s - loss: 0.0335 - accuracy: 0.9929 - val_loss: 0.0809 - val_accuracy: 0.9726
Epoch 3/10
782/782 - 12s - loss: 0.0073 - accuracy: 0.9990 - val_loss: 0.0876 - val_accuracy: 0.9704
Epoch 4/10
782/782 - 12s - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0907 - val_accuracy: 0.9716
Epoch 5/10
782/782 - 12s - loss: 7.4302e-04 - accuracy: 1.0000 - val_loss: 0.0974 - val_accuracy: 0.9714
Epoch 6/10
782/782 - 12s - loss: 3.8601e-04 - accuracy: 1.0000 - val_loss: 0.1003 - val_accuracy: 0.9715
Epoch 7/10
782/782 - 12s - loss: 2.1565e-04 - accuracy: 1.0000 - val_loss: 0.1058 - val_accuracy: 0.9712
Epoch 8/10
782/782 - 12s - loss: 1.2739e-04 - accuracy: 1.0000 - val_loss: 0.1098 - val_accuracy: 0.9711
Epoch 9/10
782/782 - 12s - loss: 7.7928e-05 - accuracy: 1.0000 - val_loss: 0.1145 - val_accuracy: 0.9710
Epoch 10/10
782/782 - 12s - loss: 4.8370e-05 - accuracy: 1.0000 - val_l

## Test Model

In [ ]:
sentence = "That's a really honest answer!"
sequence = tokenizer.texts_to_sequences([sentence])
sequence_padded = pad_sequences(sequence, maxlen=max_length, truncating=trunk_type)
prediction = model.predict(sequence_padded)
print(prediction)
print(prediction[0])

[[0.83473796]]
[0.83473796]


In [ ]:
if prediction[0] > 0.5:
    print("This comment is positive")
elif prediction[0] == 0.5:
    print("This comment is neutral")
else:
    print("This commment is negative")

This comment is positive


## Save The Model

Before loading the model into Tensorflow Serving, save the model in [SavedModel](https://www.tensorflow.org/guide/saved_model) format. It will create a protobuf file in a well-defined directory and will include a version number. [Tensorflow Serving](https://www.tensorflow.org/tfx/serving/serving_config) allows for the selection of the model version to be used for inference requests. Each version will be exported to a different sub-directory under the given path..

In [ ]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key

MODEL_DIR = tempfile.gettempdir()

version = 1

export_path = os.path.join(MODEL_DIR, str(version))

if os.path.isdir(export_path):
    print('\nAlready saved a model, cleaning up\n')
    !rm -r {export_path}

# This is the actual code for saving the model
model.save(export_path, save_format="tf")

print('Saved model to export_path = {}\n'.format(export_path))
!ls -l {export_path}

INFO:tensorflow:Assets written to: /tmp/1/assets


INFO:tensorflow:Assets written to: /tmp/1/assets


Saved model to export_path = /tmp/1

total 100
drwxr-xr-x 2 root root  4096 May  5 20:13 assets
-rw-r--r-- 1 root root 94093 May  5 20:13 saved_model.pb
drwxr-xr-x 2 root root  4096 May  5 20:13 variables


## Examining Saved Model
Use the command line utility saved_model_cli to look at the MetaGraphDefs and SignatureDefs in our SavedModel. The signature definition is defined by the input and output tensors, and stored with teh default serving key.

In [ ]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['embedding_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 250)
        name: serving_default_embedding_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0505 20:15:45.257752 140203130509184 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/pyth

# Tensorflow Serving

## Add Tensorflow Serving Distribution Url as a Package Source


In [ ]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  33443      0 --:--:-- --:--:-- --:--:-- 33443
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/

## Install Tensorflow Serving

In [ ]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 69 not upgraded.
Need to get 326 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.5.0 [326 MB]
Fetched 326 MB in 4s (83.8 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 160690 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.5.0_all.deb ...
Unpacking tensorflow-model-server (2.5.0) ...
Setting up tensorflow-model-server (2.5.0) ...


# Tensorflow Model Server

## Run the TensorFlow Model Server
Launch the TensorFlow model server with a bash script. Use the argument --bg to run the script in the background. 
- rest_api_port: The port that is used for requests.
- model_name: Create a custom name to be used in the request URL.
- model_base_path: Path to where the model is stored.

In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=helloworld \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


In [ ]:
# Viewing Server log

!tail server.log

2021-05-05 20:21:04.059174: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /tmp/1
2021-05-05 20:21:04.063469: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 64781 microseconds.
2021-05-05 20:21:04.064038: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /tmp/1/assets.extra/tf_serving_warmup_requests
2021-05-05 20:21:04.064158: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: helloworld version: 1}
2021-05-05 20:21:04.064764: I tensorflow_serving/model_servers/server_core.cc:486] Finished adding/updating models
2021-05-05 20:21:04.064858: I tensorflow_serving/model_servers/server.cc:367] Profiler service is enabled
2021-05-05 20:21:04.065407: I tensorflow_serving/model_servers/server.cc:393] Running gRPC ModelServer at 0.0.0.0:8500 ...
[

## Create JSON Object with Test Data

In [ ]:
sentence = "What a stupid thing to say and at the same time it is quite amazing to see that you can be honest in a time such as this!"
sequence = tokenizer.texts_to_sequences([sentence])
test_sentence = pad_sequences(sequence, maxlen=max_length, truncating=trunk_type)

data = json.dumps({"signature_name": "serving_default", "instances": test_sentence.tolist()})
print(data)

{"signature_name": "serving_default", "instances": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 49, 4, 377, 153, 6, 133, 3, 31, 2, 170, 56, 10, 7, 177, 478, 6, 65, 13, 23, 68, 28, 1200, 9, 4, 56, 139, 15, 12]]}


## Make Inference Request
Send a predict request as a POST to the server's REST endpoint, and pass the test data. By not specifying the version of the model to make inference with the server will give the latest model. The response wil be a JSON payload containing the predictions.

In [ ]:
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/helloworld:predict', data=data, headers=headers)

print(json_response.text)

{
    "predictions": [[0.955330074]
    ]
}


In [ ]:
prediction = json.loads(json_response.text)['predictions']
prediction = float(prediction[0][0])
print(prediction)

0.955330074


In [ ]:
if prediction > 0.5:
    print("This comment is positive")
elif prediction == 0.5:
    print("This comment is neutral")
else:
    print("This commment is negative")

This comment is positive
